# Natural Language Processing



Será usada Rede Neurais Recorrentes, aos quais são melhores para processar textos e carácteres. Faremos:


1.   Geração de carácteres
2.   Identificação de sentimento



## Sequenciando os dados
Diferente de imagens, Videos e textos precisam passar por um processo de "adaptação". Usaremos um dicionario para contar a frequência da palavra dentro do texto. Esse processo é chamado saco de palavras (bag of words). Para detectar sentimentos esse algoritimo não é muito eficaz já que ele não liga para a ordem das palavras.

'''I thought the movie was going to be bad, but it was actually amazing!'''

'''I thought the movie was going to be amazing, but it was actually bad!'''

Essas duas frases terão o mesmo peso em certos algoritmos



### Bag of Words

A frequência da palavra é contada e colocada em um dicionário, com o index da palavra. Esse algoritmo não se preocupa com a ordem das palavras no texto.  

In [1]:
vocabulario = {}  # mapeia a frequencia da palavra
word_encoding = 1
def bag_of_words(texto):
  global word_encoding

  palavras = texto.lower().split(" ")  # divide todas as palavras do texto
  bag = {}  # coloca todas as palavras e frequencia nesse dicionário 

  for palavra in palavras:
    if palavra in vocabulario:
      encoding = vocabulario[palavra]  # pega a frequencia da palavra
    else:
      vocabulario[palavra] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

texto = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(texto)
print(bag)
print(vocabulario)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


Esse algoritmo não será utilizado, porque a frase perde a sequência.

In [2]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


## Word Embedding
Esse método transforma a palavra em um vetor, mantem a ordem e frequência e também aproxima palavras com significados próximos.

## Recurrent Neural Networks (RNN's)

Esse tipo de rede neural usa o valor anterior para calcular o novo.
![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Source: https://colah.github.io/posts/2015-08-Understanding-LSTMs/*

Sendo:

**h<sub>t</sub>** saida no tempo t

**x<sub>t</sub>** entrada no tempo t

**A** camada recorrente (loop)

Por exemplo, um texto com t palavras será calculado uma palavra no H0 e será passado o calculo para H1, no qual a segunda palavra é calculada em cima do calculo da primeria e assim por diante.

Esse método faz com que ao chegar no final(Ht) a primeira palavra seja perdida.

## LSTM - Long-Short Term Memory

O RNN anterior é chamado de simples. O LSTM usa um método para acessar os valores anteriores da camada que desaparecem no RNN simples ao longo do calculo.